In [1]:
#import relavant libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
cos_mov1 = pd.read_csv('pop_key_movies.csv')

In [3]:
cos_mov1.head(10)

,level_0,Action,Documentary,Adventure,Comedy,Drama,Family,Fantasy,History,Horror,...,War,Western,Animation,Language,combined_features,title,id,popularity,index,keywords
0,0,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign silent fi...,Passage of Venus,315946,6.169,0,silent film astronomy venus the planet 19th ce...
1,21,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Documentary English horse sto...,Sallie Gardner at a Gallop,194079,7.766,1,horse stop motion black and white silent film ...
2,22,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign silent fi...,Athlete Swinging a Pick,502575,5.044,2,silent film short film
3,26,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign buffalo p...,Buffalo Running,426903,4.554,3,buffalo photography black and white short film
4,27,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,NaN,L'homme machine,585297,3.869,4,NaN
5,28,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,NaN,Child Bringing Bouquet to Woman,497867,3.515,5,NaN
6,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Animation,Foreign,Animation Foreign walking bla...,Man Walking Around a Corner,159897,4.752,6,walking black and white short film
7,51,NaN,Documentary,NaN,NaN,Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Drama Foreign musi...,Accordion Player,96882,5.004,7,musician silent film
8,52,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign silent fi...,Brighton Street Scene,355995,2.763,8,silent film lost film
9,54,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign garden ha...,Roundhay Garden Scene,16463,11.996,9,garden hat historical short film


In [74]:
#Helper Functions
def get_title_from_index(index):
    return cos_mov1[cos_mov1.index == index]["title"].values[0]

def get_index_from_title(title):
    return cos_mov1[cos_mov1.title == title]["index"].values[0]

def get_combined_features(index):
    return cos_mov1[cos_mov1.index == index]["combined_features"].values[0]

def get_id_from_index(index):
    return cos_mov1[cos_mov1.index == index]["id"].values[0]

def get_index_from_id(id):
    return cos_mov1[cos_mov1.id == id]["index"].values[0]

In [65]:
#code for CountVectorizer()

cv = CountVectorizer()
count_matrix = cv.fit_transform(cos_mov1["combined_features"])
cosine_sim = cosine_similarity(count_matrix)
movie_user_likes = "Avatar"
user_movie_id = get_index_from_title(movie_user_likes)
recommend_movies = list(enumerate(cosine_sim[user_movie_id]))
sorted_recs = sorted(recommend_movies,key=lambda x:x[1],reverse=True)
i=0
for element in sorted_recs:
		print(get_title_from_index(element[0]))
		i=i+1
		if i>10:
			break

#Code for Vectorizer

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(cos_mov1["combined_features"])
similarity = cosine_similarity(feature_vectors)
movie_user_likes = "Passage of Venus"
user_movie_id = get_index_from_title(movie_user_likes)
recommend_movies = list(enumerate(similarity[user_movie_id]))
sorted_recs = sorted(recommend_movies,key=lambda x:x[1],reverse=True)
i=0
for element in sorted_recs:
		print(get_title_from_index(element[0]))
		i=i+1
		if i>11:
			break

In [75]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(cos_mov1["combined_features"].values.astype('U'))
similarity = cosine_similarity(feature_vectors)

In [76]:
#Returns a list of top 10 recommended movies
def recSingleMovie(title, similarity):
    id = get_index_from_id(title)
    recs = list(enumerate(similarity[id]))
    sorted_recs = sorted(recs,key=lambda x:x[1],reverse=True)
    sorted_recs.pop(0) #remove the movie itself from the list
    i=0
    top_recs = []
    for element in sorted_recs:
        top_recs.append(get_id_from_index(element[0]))
        i=i+1
        if i>9:
            break
    return top_recs

In [77]:
# Test recSingleMovie Function
# single = recSingleMovie(315946, similarity)
# for x in single:
#     print(x)

49278
90787
43135
355764
104852
129436
231113
152457
67319
27625


In [78]:
#returns a list of recommended movies from a list of liked movies
def recListMovie(liked_list):
    movie_recs = []
    for i in range(len(liked_list)):
        #recSingleMovie returns a list of recommendations for a single movie
        single_movie_recs = recSingleMovie(liked_list[i], similarity)
        k=0
        for j in range(len(single_movie_recs)):
            pos = k*(i+1)
            # check if movie is in list
            if single_movie_recs[j] not in movie_recs:
                movie_recs.insert(pos, single_movie_recs[j])
                # if a new movie is added, update position
                k=k+1
    return movie_recs
        

In [80]:
# Test recListMovie function
liked_list = [315946, 194079, 315946]
movie100 = recListMovie(liked_list)
for x in movie100:
    print(x)

53406
49278
50902
90787
53402
43135
50922
355764
50906
104852
53384
129436
53405
231113
53254
152457
53403
67319
50854
27625


In [3]:
cos_mov1.head()

,level_0,Action,Documentary,Adventure,Comedy,Drama,Family,Fantasy,History,Horror,...,War,Western,Animation,Language,combined_features,title,id,popularity,index,keywords
0,0,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign silent fi...,Passage of Venus,315946,6.169,0,silent film astronomy venus the planet 19th ce...
1,21,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Documentary English horse sto...,Sallie Gardner at a Gallop,194079,7.766,1,horse stop motion black and white silent film ...
2,22,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign silent fi...,Athlete Swinging a Pick,502575,5.044,2,silent film short film
3,26,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign buffalo p...,Buffalo Running,426903,4.554,3,buffalo photography black and white short film
4,27,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,NaN,L'homme machine,585297,3.869,4,NaN


In [20]:
cos_mov = cos_mov1[cos_mov1['popularity'] > 9]

In [21]:
cos_mov.shape

(4413, 26)

In [23]:
cos_mov.head()

,level_0,Action,Documentary,Adventure,Comedy,Drama,Family,Fantasy,History,Horror,...,War,Western,Animation,Language,combined_features,title,id,popularity,index,keywords
9,54,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign garden ha...,Roundhay Garden Scene,16463,11.996,9,garden hat historical short film
152,2382,NaN,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy English moon based on ...,A Trip to the Moon,775,17.510,152,moon based on novel or book satire astronomer ...
377,9244,NaN,NaN,NaN,Comedy,Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy Drama English ku klux k...,The Birth of a Nation,618,11.092,377,ku klux klan southern usa army racist silent f...
425,10703,NaN,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy English naivety intole...,Intolerance: Love's Struggle Throughout the Ages,3059,9.374,425,naivety intolerance mill marriage wedding mass...
485,12672,Action,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Action Comedy Thriller English ...,Shoulder Arms,53423,10.722,485,world war i black and white silent film behind...


In [24]:
cos_mov.reset_index(drop=True)

,level_0,Action,Documentary,Adventure,Comedy,Drama,Family,Fantasy,History,Horror,...,War,Western,Animation,Language,combined_features,title,id,popularity,index,keywords
0,54,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign garden ha...,Roundhay Garden Scene,16463,11.996,9,garden hat historical short film
1,2382,NaN,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy English moon based on ...,A Trip to the Moon,775,17.510,152,moon based on novel or book satire astronomer ...
2,9244,NaN,NaN,NaN,Comedy,Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy Drama English ku klux k...,The Birth of a Nation,618,11.092,377,ku klux klan southern usa army racist silent f...
3,10703,NaN,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy English naivety intole...,Intolerance: Love's Struggle Throughout the Ages,3059,9.374,425,naivety intolerance mill marriage wedding mass...
4,12672,Action,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Action Comedy Thriller English ...,Shoulder Arms,53423,10.722,485,world war i black and white silent film behind...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4408,210700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Foreign undercover drug traf...,The Mod Squad,16379,10.048,36532,undercover drug trafficking undercover cop
4409,210734,NaN,NaN,Adventure,Comedy,NaN,Family,Fantasy,NaN,NaN,...,NaN,NaN,Animation,English,Adventure Comedy Family Fantasy A...,Esther,2700,10.056,36542,bible christian film
4410,210738,NaN,NaN,NaN,NaN,Drama,NaN,NaN,History,NaN,...,NaN,NaN,NaN,Foreign,Drama History Thriller Foreign ...,Bloodsport: The Dark Kumite,36693,9.296,36545,martial arts fistfight sports mixed martial ar...
4411,210739,NaN,NaN,NaN,Comedy,Drama,NaN,Fantasy,NaN,NaN,...,NaN,NaN,NaN,Foreign,Comedy Drama Fantasy Foreign hi...,10 Things I Hate About You,4951,44.982,36546,high school deception based on play or musical...


In [28]:
cos_mov.shape

(4413, 25)

In [26]:
cos_mov = cos_mov.drop(["index"], axis=1)

In [29]:
cos_mov["index"] = list(range(4413))

In [30]:
cos_mov.head()

,level_0,Action,Documentary,Adventure,Comedy,Drama,Family,Fantasy,History,Horror,...,War,Western,Animation,Language,combined_features,title,id,popularity,keywords,index
9,54,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Foreign,Documentary Foreign garden ha...,Roundhay Garden Scene,16463,11.996,garden hat historical short film,0
152,2382,NaN,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy English moon based on ...,A Trip to the Moon,775,17.510,moon based on novel or book satire astronomer ...,1
377,9244,NaN,NaN,NaN,Comedy,Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy Drama English ku klux k...,The Birth of a Nation,618,11.092,ku klux klan southern usa army racist silent f...,2
425,10703,NaN,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Comedy English naivety intole...,Intolerance: Love's Struggle Throughout the Ages,3059,9.374,naivety intolerance mill marriage wedding mass...,3
485,12672,Action,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Action Comedy Thriller English ...,Shoulder Arms,53423,10.722,world war i black and white silent film behind...,4


In [31]:
cos_mov.to_csv('pop_key_movies4500.csv', index=False)